In [1]:
#Z3 Sudoku

import z3

n=9
m=int(n**(0.5))

S=[[[ z3.Bool('S_{}_{}_{}'.format(i,j,k)) for k in range(n)] for j in range(n)] for i in range(n)]


### Making Functions

## For 4x4
#Only 1 True out of 4 Variable
def four_constraint(x):
    solver.add(z3.Or(S[x[0][0]][x[0][1]][x[0][2]],S[x[1][0]][x[1][1]][x[1][2]],S[x[2][0]][x[2][1]][x[2][2]],S[x[3][0]][x[3][1]][x[3][2]]))
    for i in range(0,3):
        for j in range(i+1,4):
            solver.add(z3.Not(z3.And(S[x[i][0]][x[i][1]][x[i][2]],S[x[j][0]][x[j][1]][x[j][2]])))
    
#Keeping Value at i,j
def four_value(x,i,j):
    for k in range(0,4):
        temp=x[k]
        if(temp==0):
            solver.add(z3.Not(S[i][j][k]))
        else:
            solver.add(S[i][j][k])

def prec_four_value(num,i,j):
    x=[0,0,0,0]
    x[num-1]=1
    four_value(x,i,j)

## For 9x9
#Only 1 True out of 9 Variable 
def nine_constraint(x):
    solver.add(z3.Or(S[x[0][0]][x[0][1]][x[0][2]],S[x[1][0]][x[1][1]][x[1][2]],S[x[2][0]][x[2][1]][x[2][2]],S[x[3][0]][x[3][1]][x[3][2]],S[x[4][0]][x[4][1]][x[4][2]],S[x[5][0]][x[5][1]][x[5][2]],S[x[6][0]][x[6][1]][x[6][2]],S[x[7][0]][x[7][1]][x[7][2]],S[x[8][0]][x[8][1]][x[8][2]]))
    for i in range(0,8):
        for j in range(i+1,9):
            solver.add(z3.Not(z3.And(S[x[i][0]][x[i][1]][x[i][2]],S[x[j][0]][x[j][1]][x[j][2]])))


#Keeping Value at i,j
def nine_value(x,i,j):
    for k in range(0,9):
        temp=x[k]
        if(temp==0):
            solver.add(z3.Not(S[i][j][k]))
        else:
            solver.add(S[i][j][k])

def add_nine(num,i,j):
    x=[0,0,0,0,0,0,0,0,0]
    x[num-1]=1
    nine_value(x,i,j)


###Finding All Tuples

total_tuples=[]


##Checking for Rows/Columns/In-Lines
for i in range(0,n):
    for j in range(0,n):
        temp1=[]
        temp2=[]
        temp3=[]
        for k in range(0,n):
            temp1.append([i,j,k])   #in-line check
            temp2.append([i,k,j])   #row check
            temp3.append([k,j,i])   #col check
        total_tuples.append(temp1)
        total_tuples.append(temp2)
        total_tuples.append(temp3)

##Checking For Grids
y=[]
for a in range(0,m):
    for b in range(0,m):
        t_a=m*a
        t_b=m*b
        temp7=[]
        for i in range(0,m):
            for j in range(0,m):
                temp7.append([i+t_a,j+t_b])
        y.append(temp7)

for i in range(0,n):
    temp4=y[i]
    temp5=[]
    for j in range(0,n):
        temp6=[]
        for k in range(0,n):
            temp6.append([temp4[k][0],temp4[k][1],j])
        temp5.append(temp6)
    total_tuples+=temp5


##All valid Tuples are formed        
cnt=0
for i in total_tuples:
    cnt+=1
    #print(cnt,i)


###Creating Solver

solver=z3.Solver()

        
##Inputing Values
value=[ 'xxxxxx2xx',
        'x8xxx7x9x',
        '6x2xxx5xx',
        'x7xx6xxxx',
        'xxx9x1xxx',
        'xxxx2xx4x',
        'xx5xxx6x3',
        'x9x4xxx7x',
        'xx6xxxxxx',
    ]

for i in range(n):
    for j in range(n):
        if(value[i][j]!='x'):
            t=int(value[i][j])
            #print(t,i,j)
            add_nine(t,i,j)

##Asserting Constraints
for i in range(len(total_tuples)):
    temp=total_tuples[i]
    nine_constraint(temp)


#print(solver)


print("Sudoku is")
for i in range(n):
    for j in range(n):
        print(value[i][j],end=" ")
    print(" ")
            

print(" ")

res = solver.check()
if res == z3.sat:
    print("Solution is:")
    M=solver.model()
    for i in range(n):
        for j in range(n):
            t=[]
            for k in range(n):
                t.append(M.evaluate(S[i][j][k]))
            print(1+t.index(True),end=" ")
        print(" ")


Sudoku is
x x x x x x 2 x x  
x 8 x x x 7 x 9 x  
6 x 2 x x x 5 x x  
x 7 x x 6 x x x x  
x x x 9 x 1 x x x  
x x x x 2 x x 4 x  
x x 5 x x x 6 x 3  
x 9 x 4 x x x 7 x  
x x 6 x x x x x x  
 
Solution is:
9 5 7 6 1 3 2 8 4  
4 8 3 2 5 7 1 9 6  
6 1 2 8 4 9 5 3 7  
1 7 8 3 6 4 9 5 2  
5 2 4 9 7 1 3 6 8  
3 6 9 5 2 8 7 4 1  
8 4 5 7 9 2 6 1 3  
2 9 1 4 3 6 8 7 5  
7 3 6 1 8 5 4 2 9  
